# Imports

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

# Methods

In [2]:
def CI_via_bootstrap(data_df, method, B=1000, alpha=0.05):
    results = np.zeros(B)
    N = len(data_df)
    for i in range(B):
        sample = data_df.sample(n=N, replace=True)
        results[i] = method(sample)

    return np.mean(results), np.std(results), np.quantile(results, ([alpha/2, 1-alpha/2]))

In [3]:
def wilcoxon(data_df):
    data_df['rank'] = data_df['salary'].rank()

    # consts in the test
    n = len(data_df[data_df['T'] == 1])
    N = len(data_df)
    m = N-n

    # expectation and variance
    expected = n*(N + 1) / 2
    var = m*n*(N + 1) / 12

    # test statistics
    W_s = data_df[data_df['T'] == 1]['rank'].sum()
    T = (W_s - expected) / var**0.5
    z_quant = scipy.stats.norm.ppf(0.95)

    if T > z_quant:
        print("Samples that were treated are statistically bigger")

    elif T < -z_quant:
        print("Samples that were treated are statistically smaller")

### IPW

In [19]:
def ipw_ATT(data_df_input):
    data_df = data_df_input.copy()
    Y = data_df['salary'].to_numpy()
    X = data_df.drop(columns=['salary'])
    N = len(X)
    T = X['T'].to_numpy()
    model_covariates = list(data_df.columns[~data_df.columns.isin(['T', 'salary'])])
    # model = LogisticRegression(solver='liblinear')
    model = LogisticRegression()
    model = model.fit(data_df[model_covariates], data_df["T"])
    prop_score = model.predict_proba(data_df[model_covariates]).T[1]
    one = np.ones(N)
    prop_div_prop = prop_score / (one - prop_score)
    first = np.sum(T * Y) / T.sum()
    second = np.sum((one - T) * Y * prop_div_prop) / np.sum((one - T) * prop_div_prop)
    return first - second

def ipw_ATE(data_df):
    model_covariates = list(data_df.columns[~data_df.columns.isin(['T', 'salary'])])
    model = LogisticRegression(solver='liblinear')
    model = model.fit(data_df[model_covariates], data_df["T"])
    treated = data_df[data_df['T'] == 1].reset_index(drop=True)
    untreated = data_df[data_df['T'] == 0].reset_index(drop=True)
    
    propensity_treated = model.predict_proba(treated[model_covariates]).T[1]
    propensity_untreated = model.predict_proba(untreated[model_covariates]).T[1]

    sum1 = (treated['salary'] / propensity_treated).sum()
    sum2 = (untreated['salary'] / (1 - propensity_untreated)).sum()
    return (sum1 - sum2) / len(data_df) 

### S-Learner

In [20]:
def s_learner_ATT(data_df):
    Y = data_df['salary']
    X = data_df.drop(columns=['salary'])
    model = LinearRegression()
    model.fit(X, Y)

    treated_X = X[X['T'] == 1]
    N = len(treated_X)

    first = model.predict(treated_X)
    untreated_X = treated_X.copy()
    untreated_X['T'].replace(1, 0, inplace=True)
    second = model.predict(untreated_X)
    return np.sum(first-second) / N

def s_learner_ATE(data_df):
    Y = data_df['salary']
    X = data_df.drop(columns=['salary'])
    
    model = LinearRegression()
    model.fit(X, Y)
    N = len(X)

   

    X_0 = X.copy()
    X_1 = X.copy()
    X_0['T'] = 0
    X_1['T'] = 1

    first = model.predict(X_1)
    second = model.predict(X_0)
    return np.sum(first - second) / N

### T-Learner

In [6]:
def t_learner_ATT(data_df):
    untreated_data = data_df[data_df['T'] == 0]
    Y_0 = untreated_data['salary']
    X_0 = untreated_data.drop(columns=['salary'])
    model_0 = LinearRegression()
    model_0.fit(X_0, Y_0)

    treated_data = data_df[data_df['T'] == 1]
    Y_1 = treated_data['salary']
    X_1 = treated_data.drop(columns=['salary'])
    model_1 = LinearRegression()
    model_1.fit(X_1, Y_1)

    N = len(X_1)
    first = model_1.predict(X_1)
    second = model_0.predict(X_1)
    return np.sum(first-second) / N  

def t_learner_ATE(data_df):
    untreated_data = data_df[data_df['T'] == 0]
    Y_0 = untreated_data['salary']
    X_0 = untreated_data.drop(columns=['salary'])
    model_0 = LinearRegression()
    model_0.fit(X_0, Y_0)

    treated_data = data_df[data_df['T'] == 1]
    Y_1 = treated_data['salary']
    X_1 = treated_data.drop(columns=['salary'])
    model_1 = LinearRegression()
    model_1.fit(X_1, Y_1)
  
    N = len(data_df)
    first = model_1.predict(data_df.drop(columns=['salary']))
    second = model_0.predict(data_df.drop(columns=['salary']))
    return np.sum(first-second) / N

### Matching

In [7]:
def matching_ATT(data_df):
    untreated_data = data_df[data_df['T'] == 0]
    Y_0 = untreated_data['salary']
    X_0 = untreated_data.drop(columns=['salary'])

    treated_data = data_df[data_df['T'] == 1]
    Y_1 = treated_data['salary']
    X_1 = treated_data.drop(columns=['salary'])

    model = KNeighborsRegressor(n_neighbors=1)
    model.fit(X_0, Y_0)

    pred = model.predict(X_1)
    return np.sum(Y_1 - pred) / len(X_1) 

def matching_ATE(data_df):
    untreated_data = data_df[data_df['T'] == 0]
    Y_0 = untreated_data['salary']
    X_0 = untreated_data.drop(columns=['salary'])

    treated_data = data_df[data_df['T'] == 1]
    Y_1 = treated_data['salary']
    X_1 = treated_data.drop(columns=['salary'])

    model_0 = KNeighborsRegressor(n_neighbors=1)
    model_0.fit(X_0, Y_0)

    model_1 = KNeighborsRegressor(n_neighbors=1)
    model_1.fit(X_1, Y_1)

    ITE_1 = model_0.predict(X_1)
    ITE_0 = model_1.predict(X_0)
    
    N = len(data_df)

    return (np.sum(Y_1 - ITE_1) + np.sum(ITE_0 - Y_0)) / N

In [8]:
def preproccesing(data_df):
    data_df = data_df.drop(columns=['_id'])
    data_df['T'] = data_df.apply(lambda row: 1 if row['T'] else 0, axis=1)
    data_df['WS'] = data_df['WS/48']
    data_df['FG'] = data_df['FG%']
    data_df['TS'] = data_df['TS%']
    drop_cols = ['TS%', 'FG%', 'WS/48']
    for col in list(data_df.columns):
        if ' ' in col:
            new_name = col.replace(' ','_')
            if '/' in new_name:
                new_name = new_name.replace('/','_')
            data_df[new_name] = data_df[col]
            drop_cols.append(col)
    data_df = data_df.drop(columns=drop_cols)
    return data_df

# Results

In [9]:
def ATT_method(data_df, method, method_as_string, B=1000):
    a1 = method(data_df)
    mean, std ,ci = CI_via_bootstrap(data_df, method, B)
    print(f"{method_as_string} ATT for our data: {round(a1, 6)}. The CI for the data is {ci}.")
    print(f"{method_as_string} ATT - The mean of the bootstrap {mean} and the std is {std}.")
   

def ATE_method(data_df, method, method_as_string, B=1000):
    a1 = method(data_df)
    mean, std, ci = CI_via_bootstrap(data_df, method, B)
    print(f"{method_as_string} ATE - The mean of the bootstrap {mean} and the std is {std}.")
    print(f"{method_as_string} ATE for our data: {round(a1, 6)}. The CI for the data is {ci}")
    print('\n')

In [10]:
methods_as_string = ['IPW', 'S-Learner', 'T-Learner', 'Matching']

att_methods = [ipw_ATT, s_learner_ATT, t_learner_ATT, matching_ATT]
ate_methods = [ipw_ATE, s_learner_ATE, t_learner_ATE, matching_ATE]

In [11]:
data = preproccesing(pd.read_csv('avg_treat_salary_5_to_15.csv'))

In [12]:
wilcoxon(data)

Samples that were treated are statistically bigger


In [17]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
for name, att_method in zip(methods_as_string, att_methods):
    ATT_method(data, att_method, name, B=1000)

IPW ATT for our data: 250303.294953. The CI for the data is [ 539214.5011326 1004897.0585787].
IPW ATT - The mean of the bootstrap 790348.5624649852 and the std is 117593.5566466767.
[-3.70105545e+03 -5.32651009e+02 -3.45571819e+03 -1.94348671e+02
  7.94343373e+02 -5.30315500e+02  1.06469259e+04  1.69050493e+05
  3.80570990e+02  1.88925059e+04  5.01634108e+04 -1.03909279e+06
  3.29200324e+05  1.60923112e+05  1.86471223e+04 -9.86620395e+04
 -1.14987984e+05 -7.00946225e+04  5.80126562e+04 -7.96649282e+04
  4.92333269e+03 -3.45141581e+04  3.63330598e+04  1.16290263e+05
 -5.69735502e+04  5.01453859e+03  5.01430979e+04  5.51433391e+04
 -5.19301631e+04  1.98685410e+05 -5.44631838e+04 -1.13054655e+05
  1.58890318e+05  6.42137308e+04 -3.86589177e+04 -2.42588631e+05
  4.85212048e+03  9.39347160e+03  1.90279146e+04 -3.59624259e+03
 -1.08818766e+05 -4.89336846e+04  3.67271469e+05 -8.06564305e+04
  4.09754438e+03 -3.27759087e+05 -1.12912446e+03  5.63069722e+04
  8.61188537e+04  1.00369007e+05  2.5

In [14]:
for name, ate_method in zip(methods_as_string, ate_methods):
    ATE_method(data, ate_method, name, B=1000)

IPW ATE - The mean of the bootstrap 320723.7314494399 and the std is 125620.09530985783.
IPW ATE for our data: 293577.075806. The CI for the data is [ 72617.39515738 560517.22927942]


S-Learner ATE - The mean of the bootstrap 16490.984352720956 and the std is 43168.09835401589.
S-Learner ATE for our data: 18892.505901. The CI for the data is [-70075.80588211 101598.6138617 ]


T-Learner ATE - The mean of the bootstrap 58543.59302304366 and the std is 42431.99010244501.
T-Learner ATE for our data: 59240.295349. The CI for the data is [-22508.69493222 140942.56255507]


Matching ATE - The mean of the bootstrap 335284.41881691304 and the std is 91456.2519544121.
Matching ATE for our data: 271467.864677. The CI for the data is [162936.03108354 513141.38388058]


